In [2]:
def get_top_user_info():
    import pandas as pd
    import requests
    import json
    import os
    from pprint import pprint
    import time

    # Import API key
    from api_keys import g_key

    channel_input=os.path.join('Resources','Output Data','channel_scrapped_details.csv')

    channel_input_df = pd.read_csv(channel_input)

    id_list = channel_input_df['Channel_id'].to_list()

    channel_Details_df = pd.DataFrame({'Channel_id':[],
                            'Channel_title':[],
                            'Created_date':[],
                            'Video_count':[],
                            'Views_count':[],
                            'Subscription_count':[],
                            'Subscription_count_hidden':[],
                            'Country':[]})
    count = 0
    for id in id_list:

        try:
            base_url = f'https://youtube.googleapis.com/youtube/v3/channels?part=snippet&part=statistics&id={id}&key={g_key}'
            resp = requests.get(base_url)
            results = resp.json()

            title = results['items'][0]['snippet']['title']
            created = results['items'][0]['snippet']['publishedAt']
            sub_count_stat = results['items'][0]['statistics']['hiddenSubscriberCount']
            if sub_count_stat == True:
                sub_count = 0
            else:
                sub_count_stat = False
                sub_count = results['items'][0]['statistics']['subscriberCount']
            video_cnt = results['items'][0]['statistics']['videoCount']
            view_count = results['items'][0]['statistics']['viewCount']
            country = results['items'][0]['snippet']['country']

            channel_Details_df = channel_Details_df.append({'Channel_id':id,
                                    'Channel_title':title,
                                    'Created_date':created,
                                    'Video_count':video_cnt,
                                    'Views_count':view_count,
                                    'Subscription_count':sub_count,
                                    'Subscription_count_hidden':str(sub_count_stat),
                                    'Country':country},ignore_index = True)

        except Exception as e:
            print(e)
            channel_Details_df = channel_Details_df.append({'Channel_id':id,
                                    'Channel_title':title,
                                    'Created_date':created,
                                    'Video_count':video_cnt,
                                    'Views_count':view_count,
                                    'Subscription_count':sub_count,
                                    'Subscription_count_hidden':str(sub_count_stat),
                                    'Country':'NA'},ignore_index = True)
            print(f'Details of {id} channels are not extracted successfully')

        print(f'completed {count}')
        count += 1
        time.sleep(2)
        print(f'Details of {id} channels are extracted successfully')
          
    return channel_Details_df